Select nep.txt from UNEP.txt

In [ ]:
import numpy as np

symbol_to_number = {'Ag': 0, 'Al': 1, 'Au': 2, 'Cr': 3, 'Cu': 4, 'Mg': 5, 'Mo': 6, 'Ni': 7, 'Pb': 8, 'Pd': 9, 'Pt': 10, 'Ta': 11, 'Ti': 12, 'V': 13, 'W': 14, 'Zr': 15}
select_symbol = ['W', 'Ta', 'V', 'Cr']
num_symbol = len(select_symbol)
select_number = [symbol_to_number[symbol] for symbol in select_symbol]

nep = np.loadtxt('unep.txt')
n_max = np.array([6, 4])
basis_size = np.array([12, 8])
neuron = 80
N_des = n_max[0] + 1 + (n_max[1] + 1) * 5
N_ann_one = (N_des + 2) * neuron
N_c_radial = (n_max[0] + 1) * (basis_size[0] + 1) 
N_c_angular = (n_max[1] + 1) * (basis_size[1] + 1)
N_16 = N_ann_one * 16 + 1 + N_c_radial * 16 * 16 + N_c_angular * 16 * 16 + N_des

# Open file for writing
with open("nep.txt", "w") as fid:
    offset = 0
    for n in range(16):
        for m in range(N_ann_one):
            if n in select_number:
                fid.write(f"{nep[offset]:15.7e}\n")
            offset += 1

    fid.write(f"{nep[offset]:15.7e}\n")
    offset += 1

    for m in range(N_c_radial + N_c_angular):
        for n1 in range(16):
            for n2 in range(16):
                if n1 in select_number and n2 in select_number:
                    fid.write(f"{nep[offset]:15.7e}\n")
                offset += 1

    for n in range(N_des):
        fid.write(f"{nep[offset]:15.7e}\n")
        offset += 1

Fit ZBL Parameters.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from wizard.io import read_xyz

def quadratic(r, p1, p2, p3, p4, p5, p6):
    Z1 = 74
    Z2 = 74
    a = 0.46848 / (Z1 ** 0.23 + Z2 ** 0.23) 
    A = 14.399645 * Z1 * Z2
    x = r / a
    return A / r * (p1 * np.exp(-p2 * x) + p3 * np.exp(-p4 * x) + p5 * np.exp(-p6 * x)) 

dimer = read_xyz('dimer.xyz')
r = []
y = []
p0 = [0.5, 0.5, 0.5, 0.5, 0.5, 0.55]
bounds = ([0.1, 0.1, 0.1, 0.1, 0.1, 0.1], [1, 1, 1, 1, 1, 1])
for atoms in dimer:
    r.append(np.linalg.norm(atoms[0].position - atoms[1].position))
    y.append(atoms.info['energy'])
r = np.array(r)
y = np.array(y)

popt, pcov = curve_fit(quadratic, r, y, p0=p0, bounds= bounds, maxfev=100000)
print(popt)
plt.scatter(r, y)
plt.plot(r, quadratic(r, *popt), color='red')
plt.show()